In [3]:
import pandas as pd

C:\Users\saffana\AppData\Local\Temp\ipykernel_20668\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
df = pd.read_csv("./tweet_labeled.csv")

OSError: [Errno 22] Invalid argument: './tweet_labeled.csv'

### Preprocessing

In [7]:
%pip install Sastrawi

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install gensim


   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 1.2 MB/s eta 0:00:21
   ---------------------------------------- 0.1/24.0 MB 1.2 MB/s eta 0:00:21
   ---------------------------------------- 0.2/24.0 MB 1.1 MB/s eta 0:00:23
   ---------------------------------------- 0.2/24.0 MB 1.1 MB/s eta 0:00:22
   ---------------------------------------- 0.2/24.0 MB 1.1 MB/s eta 0:00:22
    --------------------------------------- 0.3/24.0 MB 1.2 MB/s eta 0:00:21
    --------------------------------------- 0.4/24.0 MB 969.8 kB/s eta 0:00:25
    --------------------------------------- 0.4/24.0 MB 1.0 MB/s eta 0:00:24
    --------------------------------------- 0.4/24.0 MB 949.4 kB/s eta 0:00:25
    --------------------------------------- 0.5/24.0 MB 983.0 kB/s eta 0:00:24
    --------------------------------------- 0.5/24.0 MB 1.0 MB/s eta 0:00:23
   - -

In [4]:
# Daftar kolom yang akan dihapus
columns_to_drop = ['conversation_id_str', 'created_at', 'favorite_count', 'id_str', 'quote_count', 
                   'reply_count', 'retweet_count', 'tweet_url', 'username']

# Menghapus kolom-kolom yang tidak diinginkan
df = df.drop(columns=columns_to_drop)

In [5]:
df

,full_text,label
0,@livyyrenata You bolos aja ke jepang ya kali k...,positif
1,@livyyrenata with me please mau ketemu prim te...,positif
2,@convomfs pernah crush in @livyyrenata,positif
3,hayoyooooo rep = kamu bersedia menjadi pacarku...,positif
4,@sleezechit @livyyrenata udh sering di mention...,positif
...,...,...
495,@livyyrenata @howtodresvvell Livy... JANGAN NG...,negatif
496,@livyyrenata @howtodresvvell WKWKWKWK bener si...,negatif
497,@livyyrenata @howtodresvvell org fakir malah n...,negatif
498,Bukan gatau malu doang dia gak lebih dari seon...,negatif


In [6]:
import re
# from deep_translator import GoogleTranslator
from nltk.corpus import  stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from gensim import corpora

In [1]:
import re

def clean_twitter_text(text):
    # Identifikasi dan hapus mention
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    # Tokenisasi teks
    tokens = text.split()
    # Gabungkan kembali token menjadi teks
    clean_text = ' '.join(tokens)
    return clean_text

# Contoh penggunaan
tweet = df[full_text]
cleaned_tweet = clean_twitter_text(tweet)
print(cleaned_tweet)


Halo dan , apa kabar? #Twitter #NLP


In [6]:
# Fungsi untuk menghapus kata-kata dengan satu huruf
def remove_tag(text):
    text = text.lower()
    words = text.split()
    filtered_words = [word for word in words if not word.startswith('#')]  # Menghapus kata-kata yang dimulai dengan '#'
    filtered_words = [word for word in filtered_words if not word.startswith('@')]  # Menghapus kata-kata yang dimulai dengan '#'
    filtered_words = [word for word in filtered_words if not word.startswith('http')]  # Menghapus kata-kata yang dimulai dengan '#'
    filtered_words = [word for word in filtered_words if not word.startswith('https')]  # Menghapus kata-kata yang dimulai dengan '#'
    return ' '.join(filtered_words)

# Fungsi untuk menghapus tanda baca
def remove_punctuation(text):
    text = text.lower()
    hasil = re.sub(r'[^\w\s]', '', text)
    return hasil

def ubah_angka(text):
    text = text.lower()
    # Menggunakan regular expression untuk mengganti karakter alay
    teks_benar = re.sub(r'3', 'e', text)
    teks_benar = re.sub(r'4', 'a', teks_benar)
    teks_benar = re.sub(r'1', 'i', teks_benar)
    teks_benar = re.sub(r'0', 'o', teks_benar)
    return teks_benar

def remove_number(text) :
    hasil = re.sub(r'\d', '', text)
    return hasil

# Fungsi untuk menghapus tautan dari teks
def remove_links(text):
    text = text.lower()
    # Menggunakan ekspresi reguler untuk mencari dan menghapus tautan
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    return re.sub(r'https\S+|www\S+|https\S+', '', text)

# remove english stopwords function
def remove_stopwords(text):
    text = text.lower()
    stop_words = set(stopwords.words('indonesian'))
    word_tokens = text.split()
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

def stopwords_indo(text):
    text = text.lower()
    stop_factory = StopWordRemoverFactory()
    more_stopword = ['dengan', 'ia','bahwa','oleh']
    stop_factory.get_stop_words()+more_stopword
    stopword = stop_factory.create_stop_word_remover()
    return stopword.remove(text)

def stemming(text):
    # text = text.lower()
    # Fact = StemmerFactory()
    # Stemmer = Fact.create_stemmer()
    # return Stemmer.stem(text)
    ps = PorterStemmer()
    return ps.stem(text)


# Preprocessing kolom 'text'
df['full_text'] = df['full_text'].apply(remove_punctuation)
df['full_text'] = df['full_text'].apply(remove_tag)
df['full_text'] = df['full_text'].apply(ubah_angka)
df['full_text'] = df['full_text'].apply(remove_number)
df['full_text'] = df['full_text'].apply(remove_links)
df['full_text'] = df['full_text'].apply(stopwords_indo)

# Menampilkan DataFrame setelah preprocessing
df.head(2)

,full_text,label
0,livyyrenata you bolos aja jepang kali thailand...,positif
1,livyyrenata with me please mau ketemu prim ter...,positif


In [8]:
# df.to_csv("tweets_cleaned.csv")

In [11]:
# Tokenize the text in the 'full_text' column
texts = [text.split() for text in df['full_text']]

# Create a dictionary from the tokenized text
dictionary = corpora.Dictionary(texts)

# Convert the text to a Bag of Words (BoW) representation
corpus = [dictionary.doc2bow(text) for text in texts]

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

# Instantiate the vectorizer objects
countvectorizer = CountVectorizer(analyzer='word', stop_words='indonesian')
tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words='indonesian')

# Convert the documents into a matrix using CountVectorizer
count_wm = countvectorizer.fit_transform(df['full_text'])

# Convert the documents into a matrix using TfidfVectorizer
tfidf_wm = tfidfvectorizer.fit_transform(df['full_text'])

# Retrieve the terms found in the corpora
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

# Create DataFrames to display the results
df_countvect = pd.DataFrame(data=count_wm.toarray(), index=df.index, columns=count_tokens)
df_tfidfvect = pd.DataFrame(data=tfidf_wm.toarray(), index=df.index, columns=tfidf_tokens)

print("Count Vectorizer\n")
print(df_countvect)
print("\nTF-IDF Vectorizer\n")
print(df_tfidfvect)

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got 'indonesian' instead.

# prepros baru

In [12]:
#importing the necessary libraries
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [13]:
#creating a custom dictionary for emoticon replacement
emoticons = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

In [14]:
#function cleans the data by removing URLS, replacing emoticons with their sentiment, removing usernames,
#removing numbers and replacing all 3 or more consecutive letter by 2
#in the end it applies lemmatization
def clean_data(data):
    cleanData = []
    
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for text in data:
        text = text.lower()
        
        text = re.sub(urlPattern,'',text)
        for emoticon in emoticons.keys():
            text = text.replace(emoticon, "emoticon" + emoticons[emoticon])        
        text = re.sub(userPattern,'', text)        
        text = re.sub(alphaPattern, " ", text)
        text = re.sub(sequencePattern, seqReplacePattern, text)
        
        words = ''
        for word in text.split():

            if len(word)>1:
                word = wordLemm.lemmatize(word)
                words += (word+' ')
                
        cleanData.append(words)
        
    return cleanData

In [15]:
df['full_text'] = clean_data(df['full_text'])

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\saffana/nltk_data'
    - 'c:\\Users\\saffana\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'c:\\Users\\saffana\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'c:\\Users\\saffana\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\saffana\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
